# Version 5.0: 웨이팅 핫플레이스 Hybrid RAG 버전

이 노트북은 **원본 소스 + 요약문 예시**를 모두 활용하는 Hybrid RAG 방식입니다.

## 📌 3가지 RAG 방식 비교

| 방식 | 검색 대상 | 장점 | 단점 |
|------|-----------|------|------|
| **Example-based** (v3.0) | 요약문 예시 | 일관된 스타일 | 컨텍스트 부족 |
| **Source-based** (v4.0) | 원본 소스 | 풍부한 정보 | 스타일 불안정 |
| **Hybrid** (v5.0) | 둘 다 | 최고 품질 | 토큰 비용 증가 |

## 🎯 Hybrid 접근의 이점

1. **배경 지식**: 원본 소스에서 풍부한 정보 획득
2. **스타일 일관성**: 요약문 예시에서 톤앤매너 학습
3. **최적 균형**: 정보의 깊이 + 표현의 일관성

## 🗄️ 사용 컬렉션
- `waiting_hotplace_sources` ← 원본 소스 검색 (배경 지식)
- `waiting_hotplace_examples` ← 요약문 예시 검색 (스타일) + 저장

## ⚙️ 검색 전략
- 원본 소스: `top_k=2` (핵심 정보만)
- 요약문 예시: `top_k=2` (스타일 참고)
- 총 4개 문서 활용 → 풍부하면서도 집중된 컨텍스트

## 🚀 사용 전 준비
1. `knowledge_base/source_indexer.ipynb` 실행 (원본 소스 인덱싱)
2. `main_rag.ipynb` 또는 `main_rag_source.ipynb` 실행 (요약문 예시 축적)

## ⚠️ 섹션 1-4: 초기화

In [ ]:
# 섹션 1: Package 설치
%pip install google-cloud-aiplatform google-genai python-dotenv chromadb

In [ ]:
# 섹션 2: 라이브러리 Import
import os
import json
import pandas as pd
import vertexai
from google import genai
from google.genai import types
from dotenv import load_dotenv
import chromadb
from datetime import datetime
from vertexai.language_models import TextEmbeddingModel

print("✅ 라이브러리 Import 완료")

In [ ]:
# 섹션 3: Vertex AI 초기화
load_dotenv()

PROJECT_ID = "wad-dw"
LOCATION = "us-central1"

vertexai.init(project=PROJECT_ID, location=LOCATION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

print(f"✅ Vertex AI 초기화 완료: {PROJECT_ID} ({LOCATION})")

In [ ]:
# 섹션 4: Chroma 벡터 DB 초기화 (Hybrid - 공통 벡터 DB)
try:
    # 공통 벡터 DB 경로
    chroma_client = chromadb.PersistentClient(path="../chroma_db")
    
    # 원본 소스 컬렉션 (배경 지식용)
    waiting_hotplace_sources = chroma_client.get_or_create_collection(
        name="waiting_hotplace_sources",
        metadata={"hnsw:space": "cosine"}
    )
    
    # 요약문 예시 컬렉션 (스타일 참고 + 저장용)
    waiting_hotplace_examples = chroma_client.get_or_create_collection(
        name="waiting_hotplace_examples",
        metadata={"hnsw:space": "cosine"}
    )
    
    print("✅ Chroma 벡터 DB 초기화 완료 (Hybrid Mode - 공통 벡터 DB)")
    print(f"   - 저장 위치: ../chroma_db (shop_summary/chroma_db)")
    print(f"   - knowledge_base/source_indexer.ipynb에서 최초 1회 인덱싱 필요\n")
    print(f"   📂 원본 소스 (배경 지식):")
    print(f"      - waiting_hotplace_sources: {waiting_hotplace_sources.count()}개 청크")
    print(f"\n   📂 요약문 예시 (스타일 참고):")
    print(f"      - waiting_hotplace_examples: {waiting_hotplace_examples.count()}개 요약문")
    
    # 경고
    if waiting_hotplace_sources.count() == 0:
        print("\n   ⚠️ waiting_hotplace_sources가 비어있습니다. source_indexer.ipynb를 먼저 실행하세요.")
    if waiting_hotplace_examples.count() == 0:
        print("\n   ⚠️ waiting_hotplace_examples가 비어있습니다. 스타일 참고가 없이 원본 소스만 사용됩니다.")
    
except Exception as e:
    print(f"❌ Chroma 초기화 실패: {e}")
    chroma_client = None
    waiting_hotplace_sources = None
    waiting_hotplace_examples = None

## ✅ 섹션 5: Hybrid RAG 함수 정의

In [ ]:
# ========================================
# Hybrid RAG 함수
# ========================================

def generate_embedding(text, model="text-embedding-004"):
    """텍스트를 768차원 벡터로 변환"""
    try:
        embedding_model = TextEmbeddingModel.from_pretrained(model)
        embeddings = embedding_model.get_embeddings([text])
        return embeddings[0].values
    except Exception as e:
        print(f"❌ 임베딩 생성 실패: {e}")
        return None


def retrieve_similar_sources(query_text, collection, top_k=2):
    """원본 소스 검색"""
    if not collection or collection.count() == 0:
        return []
    
    try:
        # 쿼리 임베딩 생성
        query_embedding = generate_embedding(query_text)
        if not query_embedding:
            return []
        
        # Chroma 벡터 검색
        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=min(top_k, collection.count()),
            include=["documents", "metadatas", "distances"]
        )
        
        # Chroma 결과 형식 변환
        formatted_results = []

        # results는 배치 형식: {'ids': [[...]], 'documents': [[...]], ...}
        if results['ids'] and len(results['ids'][0]) > 0:
            for i in range(len(results['ids'][0])):
                metadata = results['metadatas'][0][i]
                distance = results['distances'][0][i]
                document = results['documents'][0][i]

                # 코사인 거리를 유사도로 변환 (distance 범위: 0~2)
                # distance 0 = 동일, distance 2 = 정반대
                # similarity = 1 - (distance / 2)                
                similarity = 1.0 - (distance / 2.0)
                
                formatted_results.append({
                    "shop_name": metadata.get("shop_name"),
                    "source_type": metadata.get("source_type"),
                    "text": document,
                    "score": similarity
                })
        
        return formatted_results
    except Exception as e:
        print(f"⚠️ 원본 소스 검색 실패: {e}")
        return []


def retrieve_similar_examples(query_text, collection, top_k=2):
    """요약문 예시 검색"""
    if not collection or collection.count() == 0:
        return []
    
    try:
        query_embedding = generate_embedding(query_text)
        if not query_embedding:
            return []
        
        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=min(top_k, collection.count()),
            include=["documents", "metadatas", "distances"]
        )
        
        formatted_results = []
        if results['ids'] and len(results['ids'][0]) > 0:
            for i in range(len(results['ids'][0])):
                metadata = results['metadatas'][0][i]
                distance = results['distances'][0][i]
                similarity = 1.0 - (distance / 2.0)
                
                formatted_results.append({
                    "shop_name": metadata.get("shop_name"),
                    "title": metadata.get("title"),
                    "summaries": json.loads(metadata.get("summaries", "[]")),
                    "score": similarity
                })
        
        return formatted_results
    except Exception as e:
        print(f"⚠️ 요약문 예시 검색 실패: {e}")
        return []


def format_hybrid_context(sources, examples):
    """
    원본 소스 + 요약문 예시를 하이브리드 컨텍스트로 포맷
    """
    context = ""
    
    # 1. 원본 소스 섹션
    if sources:
        context += "[참고 자료 - 원본 소스]\n\n"
        
        type_mapping = {
            "michelin_review": "미쉐린 가이드",
            "blueribbon_review": "블루리본",
            "chef_interview": "셰프 인터뷰",
            "course_description": "코스 설명",
            "brand_philosophy": "브랜드 철학",
            "space_ambiance": "공간/분위기",
            "tradition_technique": "전통/기법"
        }
        
        for idx, source in enumerate(sources, 1):
            shop_name = source.get('shop_name', 'Unknown')
            source_type = source.get('source_type', 'unknown')
            type_kr = type_mapping.get(source_type, source_type)
            text = source.get('text', '')
            score = source.get('score', 0)
            
            context += f"**원본 자료 {idx}: {shop_name} - {type_kr}**\n"
            context += f"{text}\n"
            context += f"유사도: {score:.3f}\n"
            context += "─" * 80 + "\n\n"
    else:
        context += "[원본 소스 없음]\n\n"
    
    # 2. 요약문 예시 섹션
    if examples:
        context += "[작성 예시 - 요약문 스타일 참고]\n\n"
        
        for idx, example in enumerate(examples, 1):
            shop_name = example.get('shop_name', 'Unknown')
            title = example.get('title', '')
            summaries = example.get('summaries', [])
            score = example.get('score', 0)
            
            context += f"**작성 예시 {idx}: {shop_name}**\n"
            context += f"제목: {title}\n"
            context += f"요약문:\n"
            for i, summary in enumerate(summaries, 1):
                context += f"  {i}. {summary}\n"
            context += f"유사도: {score:.3f}\n"
            context += "─" * 80 + "\n\n"
    else:
        context += "[작성 예시 없음]\n\n"
    
    return context


def store_successful_example(collection, shop_seq, shop_name, collected_info, 
                            title, summaries, category="waiting_hotplace"):
    """성공한 요약문 저장"""
    if not collection:
        return False
    
    try:
        info_embedding = generate_embedding(collected_info)
        if not info_embedding:
            return False
        
        doc_id = f"{category}_{shop_seq}_{datetime.utcnow().strftime('%Y%m%d%H%M%S')}"
        
        collection.add(
            documents=[collected_info],
            embeddings=[info_embedding],
            metadatas=[{
                "shop_seq": shop_seq,
                "shop_name": shop_name,
                "category": category,
                "title": title,
                "summaries": json.dumps(summaries, ensure_ascii=False),
                "created_at": datetime.utcnow().isoformat()
            }],
            ids=[doc_id]
        )
        
        print(f"✅ 벡터 DB 저장 완료: {shop_name}")
        return True
    except Exception as e:
        print(f"❌ 저장 실패: {e}")
        return False


print("✅ Hybrid RAG 함수 정의 완료")
print("   - retrieve_similar_sources(): 원본 소스 검색")
print("   - retrieve_similar_examples(): 요약문 예시 검색")
print("   - format_hybrid_context(): 하이브리드 컨텍스트 생성 (NEW)")
print("   - store_successful_example(): 요약문 저장")

## ✅ 섹션 6-7: 매장 정보 입력

In [ ]:
# ========================================
# 매장 정보 입력 (다중 매장 모드)
# ========================================

# 1️⃣ 매장 기본 정보 (shop_seq, shop_name)
SHOPS = [
    (1, "성수연방"),
    (2, "르바게트"),
    (3, "담소사골순대"),
]

# 2️⃣ 매장별 수집된 정보 (매장명을 키로 사용)
COLLECTED_INFO = {
    "성수연방": "",
    "르바게트": "",
    "담소사골순대": "",
}

# 3️⃣ SHOP_LIST 자동 생성 (shop_seq, shop_name, collected_info 매핑)
SHOP_LIST = [
    {
        "shop_seq": seq,
        "shop_name": name,
        "collected_info": COLLECTED_INFO.get(name, "[정보 없음 - COLLECTED_INFO에 추가 필요]")
    }
    for seq, name in SHOPS
]

# 출력
print(f"📍 다중 매장 모드")
print(f"   - 총 매장 수: {len(SHOP_LIST)}개")
for idx, shop in enumerate(SHOP_LIST, 1):
    info_length = len(shop['collected_info'].strip())
    status = "✅" if info_length > 100 else "⚠️"
    print(f"   {status} {idx}. {shop['shop_name']} (seq: {shop['shop_seq']}) - {info_length:,}자")

In [ ]:
# 이 섹션은 건너뜁니다.
# 섹션 6의 COLLECTED_INFO 딕셔너리에서 이미 정보를 입력했습니다.

print("✅ 다중 매장 모드")
print("   섹션 6의 COLLECTED_INFO에 매장별 정보를 입력하세요.")
print("   입력 완료 후 바로 섹션 8을 실행하세요.")

## ✅ 섹션 8: Hybrid RAG 실행

In [ ]:
# ========================================
# Hybrid RAG 프롬프트 실행
# ========================================

system_prompt = """
당신은 트렌디하고 대중적인 맛집 소개 전문 작가입니다.

## 제공된 자료
1. **원본 자료**: 미쉐린 가이드, 블루리본, 셰프 인터뷰 등 (배경 지식 및 사실 정보)
2. **작성 예시**: 과거 성공한 요약문 (톤앤매너 및 스타일 참고)

## 작성 원칙
1. **정보의 깊이**: 원본 자료의 사실과 디테일 활용
2. **스타일 일관성**: 작성 예시의 톤앤매너 참고
3. **독창성**: 예시를 그대로 복사하지 말고 해당 매장만의 특징 표현
4. 구조: 제목 1개 + 요약 문장 3개
5. 길이: 각 문장 40-60자

## 요약 문장 구조
- **1번 문장**: 셰프 철학 / 브랜드 정체성
- **2번 문장**: 코스 구성 / 시그니처 메뉴
- **3번 문장**: 공간 특징 / 미식 경험

## 출력 형식
{{
  "shop_seq": 숫자,
  "shop_name": "매장명",
  "title": "제목 (15-30자)",
  "summaries": ["문장1", "문장2", "문장3"]
}}
"""

MODEL_NAME = "gemini-2.5-pro"
generation_config = types.GenerateContentConfig(
    temperature=0.5,
    max_output_tokens=4096,
    response_mime_type="application/json"
)

print(f"🤖 모델: {MODEL_NAME} (Hybrid RAG)\n")


def generate_summary_with_hybrid_rag(shop_seq, shop_name, collected_info):
    """
    Hybrid RAG: 원본 소스 + 요약문 예시 모두 활용
    """
    
    # 1. 원본 소스 검색 (배경 지식)
    similar_sources = retrieve_similar_sources(
        query_text=collected_info,
        collection=waiting_hotplace_sources,
        top_k=2
    )
    
    # 2. 요약문 예시 검색 (스타일 참고)
    similar_examples = retrieve_similar_examples(
        query_text=collected_info,
        collection=waiting_hotplace_examples,
        top_k=2
    )
    
    # 3. 하이브리드 컨텍스트 생성
    hybrid_context = format_hybrid_context(similar_sources, similar_examples)
    
    # 4. 프롬프트 생성
    user_prompt = f"""
    {hybrid_context}

    [생성할 매장]
    매장명: {shop_name}
    Shop Seq: {shop_seq}

    [수집된 정보]
    {collected_info}

    위 원본 자료의 정보를 바탕으로, 작성 예시의 스타일을 참고하여
    {shop_name} 매장만의 고유한 요약문을 작성하세요.
    """
    
    try:
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=f"{system_prompt}\n\n{user_prompt}",
            config=generation_config
        )
        
        result_json = json.loads(response.text)
        
        usage_metadata = response.usage_metadata if hasattr(response, 'usage_metadata') else None
        token_info = {}
        if usage_metadata:
            token_info = {
                "input_tokens": usage_metadata.prompt_token_count,
                "output_tokens": usage_metadata.candidates_token_count,
                "total_tokens": usage_metadata.total_token_count
            }
        
        return {
            "status": "success",
            "data": result_json,
            "tokens": token_info,
            "source_count": len(similar_sources),
            "example_count": len(similar_examples)
        }
    except Exception as e:
        return {
            "status": "error",
            "error_type": type(e).__name__,
            "message": str(e),
            "tokens": {},
            "source_count": len(similar_sources),
            "example_count": len(similar_examples)
        }


# 실행
all_results = []

if MODE == "single":
    print(f"🔍 [{SHOP_NAME}] Hybrid RAG 검색 중...")
    print(f"   - 원본 소스 검색 (배경 지식)")
    print(f"   - 요약문 예시 검색 (스타일 참고)\n")
    
    result = generate_summary_with_hybrid_rag(SHOP_SEQ, SHOP_NAME, collected_info)
    
    print(f"   ✅ 원본 소스: {result['source_count']}개")
    print(f"   ✅ 요약문 예시: {result['example_count']}개\n")
    print(f"📝 요약문 생성 중...\n")
    
    if result["status"] == "success":
        result_json = result["data"]
        all_results.append(result_json)
        
        tokens = result.get("tokens", {})
        if tokens:
            print(f"📊 토큰 사용량:")
            print(f"   - Input:  {tokens.get('input_tokens', 0):,} tokens")
            print(f"   - Output: {tokens.get('output_tokens', 0):,} tokens")
            print(f"   - Total:  {tokens.get('total_tokens', 0):,} tokens\n")
        
        print("✅ Hybrid RAG 요약문 생성 완료\n")
        print("=" * 80)
        print(json.dumps(result_json, ensure_ascii=False, indent=2))
        print("=" * 80)
    else:
        print(f"❌ 에러: {result['message']}")

elif MODE == "multi":
    print(f"📝 {len(SHOP_LIST)}개 매장 Hybrid RAG 실행\n")
    
    for idx, shop in enumerate(SHOP_LIST, 1):
        shop_seq = shop["shop_seq"]
        shop_name = shop["shop_name"]
        collected_info = shop["collected_info"]
        
        print(f"[{idx}/{len(SHOP_LIST)}] {shop_name}...")
        
        result = generate_summary_with_hybrid_rag(shop_seq, shop_name, collected_info)
        
        if result["status"] == "success":
            all_results.append(result["data"])
            print(f"   ✅ 성공 | 소스: {result['source_count']}개 | 예시: {result['example_count']}개")
        else:
            print(f"   ❌ 실패: {result['message']}")
    
    print(f"\n✅ 완료: {len(all_results)}개")

## ✅ 섹션 9-10: 검증 및 저장

(기존 main_rag.ipynb의 섹션 9-10과 동일)

In [ ]:
print("✅ 검증 및 저장")
print("   (기존 코드와 동일 - 필요 시 복사)")

## 🎉 완료!

## 3가지 RAG 방식 사용 가이드

### 언제 어떤 방식을 사용할까?

| 상황 | 추천 방식 | 이유 |
|------|-----------|------|
| 초기 (데이터 없음) | Source-based (v4.0) | 원본 소스만 있어도 작동 |
| 스타일 일관성 중요 | Example-based (v3.0) | 예시 학습으로 톤앤매너 통일 |
| 최고 품질 필요 | **Hybrid (v5.0)** | 정보의 깊이 + 스타일 일관성 |
| 비용/속도 고려 | Example-based (v3.0) | 가장 적은 토큰 사용 |

### 점진적 개선 경로

```
1단계: Example-based (v3.0)
   ↓
2단계: Source 데이터 수집 및 인덱싱
   ↓
3단계: Source-based (v4.0) 테스트
   ↓
4단계: Hybrid (v5.0) 최종 적용
```